In [1]:
from vivino_community_reviews import vivino_reviews, concat_reviews

import pandas as pd

In [2]:
locator_more = "//button[@tabindex='0']"
locator_recent = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

locator_review = "communityReview__reviewText--2bfLj"
locator_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"

kwargs = {
    'loc1': locator_more,
    'loc2': locator_recent, 
    'loc3': locator_review,
    'loc4': locator_date
}

In [3]:
kwargs.update({
    'end_date': '20220101',
    #'end_date': '20231101',
    #'max_scr': 10, 'time_scr': 2, 'check_idx': 4,
})

In [4]:
wines = list()
wines.append([
    'Casillero del Diablo Cabernet Sauvignon (Reserva)',
    'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
])
wines.append([
    'Yellow Tail Cabernet Sauvignon',
    'https://www.vivino.com/yellow-tail-cabernet-sauvignon/w/2549?ref=nav-search#all_reviews'
])
wines.append({
    'Roche Mazet Cuvée Spéciale Cabernet Sauvignon',
    'https://www.vivino.com/fr-roche-mazet-cuvee-speciale-cabernet-sauvignon/w/1614157#all_reviews'
})
wines.append({
    'San Pedro Gato Negro Cabernet Sauvignon',
    'https://www.vivino.com/san-pedro-central-valley-vspt-gato-negro-cabernet-sauvignon/w/3565573?ref=nav-search#all_reviews'
})
wines.append({
    'Aguirre Dos Copas Cabernet Sauvignon',
    'https://www.vivino.com/aguirre-dos-copas-cabernet-sauvignon/w/7461670?ref=nav-search#all_reviews'
})
wines.append({
    'The 7th Generation - G7 Cabernet Sauvignon',
    'https://www.vivino.com/the-7th-generation-cabernet-sauvignon/w/1193865?ref=nav-search#all_reviews'
})

"""
wines.append({
    '',
    ''
})
""" 

"\nwines.append({\n    '',\n    ''\n})\n"

In [6]:
df_reviews = pd.DataFrame()
col_rev = ['id', 'wine', 'date', 'review']

for wine_name, wine_url in wines:
    df = vivino_reviews(wine_url, wine_name, **kwargs)
    df_reviews = concat_reviews(df_reviews, df, wine_name, col_rev, save=True)

493it [20:24,  2.48s/it]


493 reviews collected.


43it [00:51,  1.20s/it]


73it [00:40,  1.82it/s]


73 reviews collected.


913it [1:20:17,  5.28s/it]


913 reviews collected.


0it [00:00, ?it/s]

ValueError: time data '' does not match format '%b %d, %Y'

In [216]:
!zip wine_reviews.zip wine*csv

  adding: wine_0.csv (deflated 67%)
  adding: wine_1.csv (deflated 57%)
